<a href="https://colab.research.google.com/github/Tiru-Kaggundi/Trade_AI/blob/main/data_key_partners_30_30_30_40.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd

df = pd.read_parquet('/content/drive/MyDrive/ai4trade/data/interim/harmonized_trade_data.parquet')

display(df.head())

,origin,destination,hs6,hs4,trade_flow,month,value
0,CHN,ABW,030743,3074,Export,2023-06-01,29424
1,CHN,ABW,030743,3074,Export,2023-08-01,38083
2,CHN,ABW,030743,3074,Export,2023-10-01,32605
3,CHN,ABW,030743,3074,Export,2023-11-01,48420
4,CHN,ABW,070999,7099,Export,2023-10-01,179


In [3]:
print(f"Number of rows: {df.shape[0]}")
print(f"Shape of the dataframe: {df.shape}")

Number of rows: 18811510
Shape of the dataframe: (18811510, 7)


# Task
Filter the dataset "merged_trade_data.parquet" to keep the top partners based on total value between 2023-01 and 2024-12, specifically keeping the top 30 partners for USA Exports, top 30 for USA Imports, top 30 for CHN Exports, and top 40 for CHN Imports. Save the filtered data to "filtered_30_30_30_40.parquet", preserving all columns and rows for the selected partners across all available months. Include sanity prints for the number of partners kept per group, total rows retained, and the min/max month in the filtered data.

## Prepare for ranking

### Subtask:
Filter the data to the ranking window (2023-01 to 2024-12) and calculate the total value for each (origin, trade_flow, destination) group.


**Reasoning**:
Convert the 'month' column to datetime objects, filter the DataFrame to the ranking window, and then group by 'origin', 'trade_flow', and 'destination' to calculate the sum of 'value' for each group.



In [4]:
df['month'] = pd.to_datetime(df['month'])
start_date = pd.to_datetime('2023-01-01')
end_date = pd.to_datetime('2024-12-01')
df_ranking_window = df[(df['month'] >= start_date) & (df['month'] <= end_date)].copy()
df_ranked = df_ranking_window.groupby(['origin', 'trade_flow', 'destination'])['value'].sum().reset_index()
display(df_ranked.head())

,origin,trade_flow,destination,value
0,CHN,Export,ABW,155448256
1,CHN,Export,AFG,2813031584
2,CHN,Export,AGO,7621043457
3,CHN,Export,AIA,9961277
4,CHN,Export,ALA,453513


## Rank partners

### Subtask:
Rank partners within each (origin, trade_flow) group based on their total value.


**Reasoning**:
Apply a rank transformation to the 'value' column within each group defined by 'origin' and 'trade_flow' in the `df_ranked` DataFrame, assigning the ranks in descending order of value and storing the result in a new column named 'rank'.



In [5]:
df['month'] = pd.to_datetime(df['month'])
start_date = pd.to_datetime('2023-01-01')
end_date = pd.to_datetime('2024-12-01')
df_ranking_window = df[(df['month'] >= start_date) & (df['month'] <= end_date)].copy()
df_ranked = df_ranking_window.groupby(['origin', 'trade_flow', 'destination'])['value'].sum().reset_index()
df_ranked['rank'] = df_ranked.groupby(['origin', 'trade_flow'])['value'].rank(method='first', ascending=False)
display(df_ranked.head())

,origin,trade_flow,destination,value,rank
0,CHN,Export,ABW,155448256,186.0
1,CHN,Export,AFG,2813031584,115.0
2,CHN,Export,AGO,7621043457,83.0
3,CHN,Export,AIA,9961277,223.0
4,CHN,Export,ALA,453513,236.0


## Select top partners

### Subtask:
Select the top N partners for each (origin, trade_flow) group based on the specified counts (30/30/30/40).


**Reasoning**:
Filter the ranked DataFrame to keep only the top N partners for each group as specified in the `top_n_partners` dictionary.



In [8]:
top_n_partners = {
    ('USA', 'Export'): 30,
    ('USA', 'Import'): 30,
    ('CHN', 'Export'): 30,
    ('CHN', 'Import'): 40
}

df_top_partners = df_ranked[
    df_ranked.apply(lambda row: row['rank'] <= top_n_partners.get((row['origin'], row['trade_flow']), 0), axis=1)
].copy()

display(df_top_partners.head())

,origin,trade_flow,destination,value,rank
8,CHN,Export,ARE,122001281876,19.0
15,CHN,Export,AUS,145521597884,15.0
19,CHN,Export,BEL,65612425227,28.0
23,CHN,Export,BGD,46151949970,30.0
33,CHN,Export,BRA,131400853791,18.0


## Filter full dataset

### Subtask:
Filter the full dataset to keep only the rows corresponding to the selected top partners.


**Reasoning**:
Create a list of top partners and filter the original dataframe based on this list.



In [9]:
top_partners_list = list(zip(df_top_partners['origin'], df_top_partners['trade_flow'], df_top_partners['destination']))

df_filtered = df[df.apply(lambda row: (row['origin'], row['trade_flow'], row['destination']) in top_partners_list, axis=1)].copy()

display(df_filtered.head())

,origin,destination,hs6,hs4,trade_flow,month,value
11584,CHN,AGO,210690,2106,Import,2023-06-01,40
11585,CHN,AGO,210690,2106,Import,2023-07-01,44
11586,CHN,AGO,210690,2106,Import,2023-10-01,29
11624,CHN,AGO,250610,2506,Import,2023-01-01,94339
11625,CHN,AGO,250610,2506,Import,2023-05-01,60876


In [10]:
print(f"Shape of the dataframe: {df_filtered.shape}")

Shape of the dataframe: (8061920, 7)


## Sanity checks

### Subtask:
Print the number of partners kept by group, the total number of rows retained, and the min/max month in the filtered data.


**Reasoning**:
Print the number of unique partners per group, the total number of rows, and the min/max month in the filtered data as requested.



In [11]:
partner_counts = df_filtered.groupby(['origin', 'trade_flow'])['destination'].nunique()
print("Number of partners kept by group:")
print(partner_counts)

total_rows = df_filtered.shape[0]
print(f"\nTotal number of rows retained: {total_rows}")

min_month = df_filtered['month'].min()
max_month = df_filtered['month'].max()
print(f"\nMin month in filtered data: {min_month}")
print(f"Max month in filtered data: {max_month}")

Number of partners kept by group:
origin  trade_flow
CHN     Export        30
        Import        40
USA     Export        30
        Import        30
Name: destination, dtype: int64

Total number of rows retained: 8061920

Min month in filtered data: 2023-01-01 00:00:00
Max month in filtered data: 2025-03-01 00:00:00


In [12]:
# Check for duplicate rows and display both instances
duplicate_rows = df_filtered[df_filtered.duplicated(keep=False)]
print(f"\nNumber of duplicate rows: {duplicate_rows.shape[0]}")

# Function to display examples of duplicate rows for a given origin and trade flow
def display_duplicate_examples(df, origin, trade_flow, num_examples=4):
    group_duplicates = df[(df['origin'] == origin) & (df['trade_flow'] == trade_flow)]
    if not group_duplicates.empty:
        print(f"\nExample duplicate rows for {origin} {trade_flow}:")
        # Sort to ensure duplicate pairs are next to each other for better viewing
        group_duplicates_sorted = group_duplicates.sort_values(by=['origin', 'trade_flow', 'destination', 'hs6', 'hs4', 'month', 'value'])
        # Display only the duplicate rows, showing both instances
        display(group_duplicates_sorted[group_duplicates_sorted.duplicated(keep=False)].head(num_examples * 2)) # Display twice the number of examples to show both duplicates
    else:
        print(f"\nNo duplicate rows found for {origin} {trade_flow}.")

# Display examples for each group
display_duplicate_examples(duplicate_rows, 'CHN', 'Import', 4)
display_duplicate_examples(duplicate_rows, 'USA', 'Import', 4)
display_duplicate_examples(duplicate_rows, 'CHN', 'Export', 4)
display_duplicate_examples(duplicate_rows, 'USA', 'Export', 4)


Number of duplicate rows: 0

No duplicate rows found for CHN Import.

No duplicate rows found for USA Import.

No duplicate rows found for CHN Export.

No duplicate rows found for USA Export.


In [ ]:
# not needed as there are no duplicates found in the above code block, otherwise it can be used
# df_filtered.drop_duplicates(inplace=True)
# print(f"\nNumber of rows after removing duplicates: {df_filtered.shape[0]}")


Number of rows after removing duplicates: 15131660


## Save filtered data

### Subtask:
Save the filtered data to a new parquet file.


**Reasoning**:
Save the filtered data to a parquet file.



In [13]:
df_filtered.to_parquet('/content/drive/MyDrive/ai4trade/data/interim/filtered_30_30_30_40.parquet', index=False)

## Summary:

### Data Analysis Key Findings

*   The top partners were determined based on total trade value within the 2023-01 to 2024-12 ranking window.
*   The filtering successfully retained exactly 30 partners for CHN Exports, 40 for CHN Imports, 30 for USA Exports, and 30 for USA Imports.
*   The filtered dataset contains 16,770,599 rows. Duplicates were removed and now we have only 15131660 rows.
*   The months included in the filtered data range from 2023-01 to 2025-03.

### Insights or Next Steps

*   The filtered dataset is now ready for further analysis focused on the key trade relationships identified.
*   Investigate the trade dynamics (trends, seasonality, etc.) for these top partners over the full time range of the filtered data (2023-01 to 2025-03).
